# Data Preprocessing Notebook

## Course: Data Visualization
## Professor: Dr. Tweneboah
## Author: Lucas Spitzer

### Import Necessary Package(s)

In [1]:
# Importing Pandas
import pandas as pd

### 1. Read the two CSV files into dataframes.

In [2]:
# Saving CSVs as dataframes
runs = pd.read_csv("data/benchmarks_runs.csv") 
versions = pd.read_csv("data/model_versions.csv")  

### 2. Join runs on versions (runs.model == versions.id).

In [3]:
# Create dataframe 
df = runs.merge(
    versions[["id", "Model", "Version release date"]],
    left_on="model",
    right_on="id",
    how="left"
).rename(columns={
    "Model": "model_display_name"
})

### 3. Select and rename the relevant fields.

In [4]:
# Create the master dataframe from the merged result and rename columns.
master_df = df[[
    "model",
    "model_display_name",
    "Version release date",
    "task",
    "Best score (across scorers)"
]].rename(columns={
    "model_display_name": "Model Name",
    "Version release date": "Version Release Date",
    "task": "Benchmark",
    "Best score (across scorers)": "Benchmark Score"
})

### 4. Map each model name substring to its corresponding organization.

In [5]:
# manually define mapping keyword structure to be utilized later
substr_to_org = {
    "DeepSeek": "DeepSeek",
    "Hermes": "Arcee AI",
    "Claude": "Anthropic",
    "GPT-3": "OpenAI",
    "GPT-4": "OpenAI",
    "o1": "OpenAI",
    "o3": "OpenAI",
    "o4": "OpenAI",
    "Gemini": "Google",
    "Llama": "Meta AI",
    "grok": "xAI",
    "Qwen": "Alibaba",
    "qwq": "Alibaba",
    "Mistral": "Mistral AI",
    "Ministral": "Mistral AI",
    "Mixtral": "Mistral AI",
    "Gemma": "Google",
    "WizardLM": "01.AI",
    "Yi": "01.AI",
    "DBRX": "Databricks",
    "Phi": "Microsoft",
    "Eurus": "CUHK Shenzhen Research Institute",
    "Tulu": "University of Washington",
}

# Define mapping function
def map_org(model_name):
    # if model_name is missing or not text, skip
    if not isinstance(model_name, str):
        return ""
    lower_name = model_name.lower()
    for sub, org in substr_to_org.items():
        if sub.lower() in lower_name:
            return org
    return ""

# Applying the fucntion to the master dataframe
master_df["Organization"] = master_df["Model Name"].apply(map_org)

### 5. Drop the deprecated model name column.

In [6]:
# Drop the irrelevent column from the dataframe
master_df = master_df.drop(columns=["model"])

### 6. Add each organization's country by joining on the organization name.

In [7]:
# Load organization dataframe and join on orgnization name
org_df = pd.read_csv("data/organizations.csv")[['Organization','Country']]
master_df = master_df.merge(org_df, on='Organization', how='left')

### 7. Remove duplicate observations from the dataframe.

In [8]:
# Drop duplicates but keep the first occurrence
master_df = master_df.drop_duplicates()

### 8. Write out the requested columns to a CSV file.

In [9]:
# Write the dataframe to a master.csv to be utilized throughout the repository.
master_df.to_csv("data/master.csv", index=False)